In [1]:
import pandas as pd

##P1 Monthly spend of each customer.*italicized text*

In [3]:
#transforming Spend
spend = pd.read_excel('/Credit-Banking_Project-1 - Copy.xlsx', sheet_name= 'Spend')

In [4]:
spend['Month'] = pd.to_datetime(spend['Month'], format='%d-%b-%y')

spend['YearMonth'] = spend['Month'].dt.to_period('M')

monthly_spend = spend.groupby(['Customer', 'YearMonth']).sum()['Amount'].reset_index()
monthly_spend['Amount'] = monthly_spend['Amount'].astype(str)

monthly_spend.to_excel('/monthly_spend.xlsx')

<ipython-input-4-5e6cb971f57f>:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_spend = spend.groupby(['Customer', 'YearMonth']).sum()['Amount'].reset_index()


##P2 Monthly repayment of each customer.

In [5]:
#Repayment
repay = pd.read_excel('/Credit-Banking_Project-1 - Copy.xlsx', sheet_name= 'Repayment')

In [6]:
repay['Month'] = pd.to_datetime(repay['Month'], format='%d-%b-%y')

repay['YearMonth'] = repay['Month'].dt.to_period('M')

monthly_repay = repay.groupby(['Customer', 'YearMonth']).sum()['Amount'].reset_index()
monthly_repay['Amount'] = monthly_repay['Amount'].astype(str)

monthly_repay.to_excel('/monthly_repay.xlsx')


<ipython-input-6-ad8c086fbb9e>:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_repay = repay.groupby(['Customer', 'YearMonth']).sum()['Amount'].reset_index()


##P3 Highest paying 10 customers.

In [7]:
monthly_repay['Amount'] = monthly_repay['Amount'].astype(float)

total_spend_per_customer = monthly_repay.groupby('Customer')['Amount'].sum()

# Get the top 10 highest paying customers
top_10_customers = total_spend_per_customer.nlargest(10)

# Display the result
print(top_10_customers)


Customer
A22    9.767171e+06
A60    9.262032e+06
A61    8.807888e+06
A40    8.805085e+06
A47    8.529826e+06
A43    8.458621e+06
A48    8.432804e+06
A41    8.374046e+06
A49    8.259841e+06
A45    8.115210e+06
Name: Amount, dtype: float64


##P4 People in which segment are spending more money.

In [8]:
w1 = pd.read_excel('/Credit-Banking_Project-1 - Copy.xlsx', sheet_name= 'Customer Acqusition')

In [9]:
# Merge the two datasets on the 'Customer' column
merged_df = pd.merge(w1, spend, on='Customer')

# Group by 'Segment' and calculate the total spending for each segment
segment_spending = merged_df.groupby('Segment')['Amount'].sum()
segment_spending= segment_spending.astype(str)
# Display the result
print(segment_spending)


Segment
Govt              67325630.90647203
Normal Salary    107707142.55799913
Salaried_MNC        63639488.740588
Salaried_Pvt      71704311.02024616
Self Employed     70975475.20559144
Name: Amount, dtype: object


##P5 Which age group is spending more money?

In [12]:

#Removing values less than 18
w1['Age'] = w1['Age'].apply(lambda x: x if x >= 18 else pd.NA)

# Step 2: Calculate mean of values greater than 18
mean_age = w1['Age'].mean()

# Step 3: Replace less than 18 with the mean
w1['Age'].fillna(mean_age, inplace=True)
w1.to_excel('/Age_fixed.xlsx')

In [13]:
# Define age bins
bins = [0, 18, 28, 38, 48, 58, 68, 78, 88]

# Create labels for each bin
labels = ['0-18', '19-28', '29-38', '39-48', '49-58', '59-68', '69-78', '79-88']

# Cut the 'Age' column into bins
w1['Age Group'] = pd.cut(w1['Age'], bins=bins, labels=labels, right=False)

# Group by 'Age Group'
grouped_by_age = w1.groupby('Age Group').size().reset_index(name='Count')

# Display the result
print(grouped_by_age)


  Age Group  Count
0      0-18      0
1     19-28     15
2     29-38     10
3     39-48     12
4     49-58     34
5     59-68     13
6     69-78     14
7     79-88      2


##P6 Which is the most profitable segment?

In [14]:
# Group by 'Segment' and calculate total spending for each segment
segment_spending = merged_df.groupby('Segment')['Amount'].sum()

# Find the most profitable segment
most_profitable_segment = segment_spending.idxmax()

# Display the result
print("The most profitable segment is:", most_profitable_segment)


The most profitable segment is: Normal Salary


##P7 In which category the customers are spending more money?

In [15]:

# Merge the datasets based on 'Customer'
merged_data = pd.merge(w1, spend, on='Customer', how='inner')

# Calculate total spending for each segment
total_spending_by_segment = merged_data.groupby('Segment')['Amount'].sum()

# Identify the segment with the highest total spending
most_spending_segment = total_spending_by_segment.idxmax()

print("Total Spending by Segment:")
print(total_spending_by_segment)
print("\nThe category of people spending more money is:", most_spending_segment)


Total Spending by Segment:
Segment
Govt             6.732563e+07
Normal Salary    1.077071e+08
Salaried_MNC     6.363949e+07
Salaried_Pvt     7.170431e+07
Self Employed    7.097548e+07
Name: Amount, dtype: float64

The category of people spending more money is: Normal Salary


##P8 Impose an interest rate of 2.9% for each customer for any due amount.

In [29]:
merged_data = pd.merge(spend, repay, on='Customer', how='left')
merged_data['due'] = merged_data['Amount_x']-merged_data['Amount_y']
interest_rate = 0.029
merged_data['Interest Amount'] = merged_data['due'] * interest_rate

# Update 'Due Amount' with interestre
merged_data['due'] += merged_data['Interest Amount']

# Display the updated DataFrame
merged_data.to_excel('/merged_data.xlsx')

In [27]:
merged_data.head(10)


,Sl No:,Customer,Month_x,Type,Amount_x,YearMonth_x,SL No:,Month_y,Amount_y,YearMonth_y,Interest Amount,due
0,1,A1,2004-01-12,JEWELLERY,473775.834869,2004-01,1,2004-01-12,341467.483588,2004-01,3836.942187,136145.293468
1,1,A1,2004-01-12,JEWELLERY,473775.834869,2004-01,2,2004-01-03,453932.861748,2004-01,575.446221,20418.419342
2,1,A1,2004-01-12,JEWELLERY,473775.834869,2004-01,3,2004-01-15,33421.924125,2004-01,12770.263412,453124.174155
3,1,A1,2004-01-12,JEWELLERY,473775.834869,2004-01,4,2004-01-25,93265.438563,2004-01,11034.801493,391545.197799
4,1,A1,2004-01-12,JEWELLERY,473775.834869,2004-01,5,2005-01-17,1627.846954,2005-01,13692.291650,485840.279565
5,1,A1,2004-01-12,JEWELLERY,473775.834869,2004-01,6,2005-02-23,368051.184468,2005-02,3066.014862,108790.665262
6,1,A1,2004-01-12,JEWELLERY,473775.834869,2004-01,7,2005-02-01,289139.440515,2005-02,5354.455436,189990.849791
7,1,A1,2004-01-12,JEWELLERY,473775.834869,2004-01,103,2004-01-12,401386.958899,2004-01,2099.277403,74488.153373
8,1,A1,2004-01-12,JEWELLERY,473775.834869,2004-01,117,2004-02-05,196724.231955,2004-02,8034.496485,285086.099399
9,1,A1,2004-01-12,JEWELLERY,473775.834869,2004-01,131,2006-04-19,207221.060169,2006-04,7730.088466,274284.863167


##P9 Monthly profit for the bank.

In [28]:
# Calculate total repayment amount
total_repayment = spend.groupby('Month')['Amount'].sum()

# Calculate total interest income for the bank
total_interest_income = repay.groupby('Month')['Amount'].sum()

# Calculate monthly profit
monthly_profit = total_interest_income - total_repayment
monthly_profit = monthly_profit.astype(str)

monthly_profit.head(15)

Month
2004-01-03    -742242.8035266288
2004-01-12     -693538.894187808
2004-01-15    162209.32940965518
2004-01-25    -4200057.038185298
2004-02-05    -1675148.877373551
2004-03-19    493073.01151401363
2004-04-06    -44331.60310711013
2004-04-25     293831.3286310071
2004-05-05    1511314.9289707877
2004-09-13    1659353.7054917878
2004-11-23    -913070.9315543938
2005-01-17    280872.60404475033
2005-02-01    2006447.5337792588
2005-02-03    -780330.8239515144
2005-02-16     2282177.473982633
Name: Amount, dtype: object